# OpenObserve Kunai - Msticpy - sigma rules conversion

## Description
This notebook helps to convert, review and transform sigma rules to openobserve queries and alerts.

It leverages python_openobserve, msticpy and sigma.

Known issue(s)
* Alert import currently not working because of backend issue

Last updated: 2025/06/01

### Installation

In [ ]:
# Only run first time to install/upgrade msticpy to latest version
# %pip install --upgrade msticpy

### Setup

In [1]:
# Check we are running Python 3.6
import sys

MIN_REQ_PYTHON = (3, 6)
if sys.version_info < MIN_REQ_PYTHON:
    print("Check the Kernel->Change Kernel menu and ensure that Python 3.6")
    print("or later is selected as the active kernel.")
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

# imports
import pandas as pd
from datetime import datetime, timedelta

# data library imports
from msticpy.data.data_providers import QueryProvider

print("Imports Complete")

Imports Complete


In [2]:
# to create alerts
import json
from python_openobserve.openobserve import OpenObserve

In [3]:
import os

# Custom Certificate Authority?
os.environ["REQUESTS_CA_BUNDLE"] = os.environ["HOME"] + "/tmp/ca-bundle-internal.pem"

In [4]:
# use custom config file?
# os.environ['MSTICPYCONFIG'] = '/path/to/msticpyconfig.yaml'

In [ ]:
# FIXME! does not get MSTICPYCONFIG...
from msticpy.config import MpConfigFile, MpConfigEdit

mpconfig = MpConfigFile()
mpconfig.load_default()
mpconfig.view_settings()
# mpedit = MpConfigEdit()
# mpedit


## Instantiating a query provider

You can instantiate a data provider for OpenObserve by specifying the credentials in connect or in msticpy config file. 
<br> If the details are correct and authentication is successful, it will show connected.

In [ ]:
openobserve_prov = QueryProvider("OpenObserve")
# openobserve_prov.connect(connection_str=<url>, user=<user>, password=<password>)
# openobserve_prov.connect()
openobserve_prov.connect(
    connection_str="https://openobserve.example.com",
    user="root@example.com",
    password="root@example.com",
)

connected with user root@example.com


## Running test OpenObserve query

In [7]:
openobserve_query = """
SELECT log_file_name,count(*) FROM "default" GROUP BY log_file_name
"""
df = openobserve_prov.exec_query(openobserve_query, days=1, verbosity=1)
df.head()

INFO: from 2025-05-31 17:21:55.960345 to 2025-06-01 17:21:55.960345, TZ UTC
'Query Time start 1748712115960345 end 1748798515960345'
{'query': {'end_time': 1748798515960345,
           'sql': '\n'
                  'SELECT log_file_name,count(*) FROM "default" GROUP BY '
                  'log_file_name\n',
           'start_time': 1748712115960345}}
DEBUG: results shape (1, 2)


,count(*),log_file_name
0,5,history.log


In [8]:
openobserve_query = """SELECT body__systemd_unit, count(*) FROM "journald" group by body__systemd_unit order by count(*) desc"""
df = openobserve_prov.exec_query(
    openobserve_query,
    start_time=datetime.now() - timedelta(days=1),
    end_time=datetime.now() - timedelta(days=0),
)
df.head()

,body__systemd_unit,count(*)
0,ssh.service,30301
1,wpa_supplicant.service,7959
2,cron.service,5103
3,init.scope,3820
4,NaN,1350


## Sigma settings

In [9]:
rulesdir = os.environ["HOME"] + "/tmp/sigma/rules/linux/network_connection"
pipeline_f = (
    os.environ["HOME"]
    + "/tmp/pySigma-backend-openobserve/sigma/pipelines/openobserve/kunai.yml"
)
table_name = "kunai"

## Sigma functions

In [10]:
import pathlib

from sigma.collection import SigmaCollection
from sigma.backends.openobserve import openobserve
from sigma.pipelines.sysmon import sysmon_pipeline
from sigma.pipelines.common import logsource_linux_process_creation
from sigma.pipelines.common import logsource_linux_file_create
from sigma.pipelines.common import logsource_linux_network_connection

from sigma.processing.resolver import ProcessingPipelineResolver

# Create the pipeline resolver
piperesolver = ProcessingPipelineResolver()
# Add pipelines
piperesolver.add_pipeline_class(sysmon_pipeline())  # Sysmon
# FIXME! AttributeError: 'LogsourceCondition' object has no attribute 'name'
# piperesolver.add_pipeline_class(logsource_linux_process_creation()) # Linux
# piperesolver.add_pipeline_class(logsource_linux_file_create()) # Linux
# piperesolver.add_pipeline_class(logsource_linux_network_connection()) # Linux
# Create a combined pipeline
combined_pipeline = piperesolver.resolve(piperesolver.pipelines)
# Instantiate backend using the combined pipeline
openobserve_backend = openobserve.openobserveBackend(combined_pipeline)

In [11]:
from subprocess import check_output
import sqlglot

In [12]:
# `sigma convert -t openobserve -p ../pySigma-backend-openobserve/sigma/pipelines/openobserve/kunai.yml ../sigma/rules/linux/process_creation/proc_creation_lnx_doas_execution.yml`
# https://github.com/SigmaHQ/sigma-cli/blob/main/sigma/cli/convert.py#L8
from sigma.exceptions import (
    SigmaError,
    SigmaPipelineNotAllowedForBackendError,
    SigmaPipelineNotFoundError,
)
from sigma.plugins import InstalledSigmaPlugins

plugins = InstalledSigmaPlugins.autodiscover()
backends = plugins.backends
pipelines = plugins.pipelines
pipeline_resolver = plugins.get_pipeline_resolver()

In [13]:
pipelines

{'sysmon': <sigma.pipelines.base.Pipeline at 0x79bd4c050980>,
 'windows-logsources': <function sigma.pipelines.windows.windows.windows_logsource_pipeline() -> sigma.processing.pipeline.ProcessingPipeline>,
 'windows-audit': <function sigma.pipelines.windows.windows.windows_audit_pipeline() -> sigma.processing.pipeline.ProcessingPipeline>}

In [14]:
pipeline_resolver

ProcessingPipelineResolver(pipelines={'sysmon': <sigma.pipelines.base.Pipeline object at 0x79bd4c050980>, 'windows-logsources': <function windows_logsource_pipeline at 0x79bd4c0934c0>, 'windows-audit': <function windows_audit_pipeline at 0x79bd4c093560>})

In [ ]:
# https://github.com/SigmaHQ/sigma-cli/blob/main/sigma/cli/convert.py#L225
try:
    processing_pipeline = pipeline_resolver.resolve(pipeline_f, None)
except SigmaPipelineNotFoundError as e:
    raise (f"The pipeline '{e}' was not found.\n")

In [15]:
# FIXME! field names not converted
for rule_f in pathlib.Path(rulesdir).iterdir():
    # print(f"Listed rule file {rule_f}")
    if not rule_f.is_file():  # or str(rule_f).endswith('.yml'):
        print(f"Skipping rule file {rule_f}")
        continue
    with open(rule_f, "r", encoding="utf-8") as f:
        # print(f"Reading rule file {rule_f}")
        rule_y = f.read()
    print(f"Processing rule file {rule_f}")
    rule = SigmaCollection.from_yaml(rule_y)
    rule_sql = openobserve_backend.convert(rule)[0]
    rule_sql = rule_sql.replace("<TABLE_NAME>", f'"{table_name}"')
    # verify sql
    try:
        sqlglot.transpile(rule_sql)
    except sqlglot.errors.ParseError as e:
        raise e
    print(f"```{rule_sql}```")

Processing rule file /home/user/tmp/sigma/rules/linux/network_connection/net_connection_lnx_crypto_mining_indicators.yml
```SELECT * FROM "kunai" WHERE DestinationHostname='pool.minexmr.com' OR DestinationHostname='fr.minexmr.com' OR DestinationHostname='de.minexmr.com' OR DestinationHostname='sg.minexmr.com' OR DestinationHostname='ca.minexmr.com' OR DestinationHostname='us-west.minexmr.com' OR DestinationHostname='pool.supportxmr.com' OR DestinationHostname='mine.c3pool.com' OR DestinationHostname='xmr-eu1.nanopool.org' OR DestinationHostname='xmr-eu2.nanopool.org' OR DestinationHostname='xmr-us-east1.nanopool.org' OR DestinationHostname='xmr-us-west1.nanopool.org' OR DestinationHostname='xmr-asia1.nanopool.org' OR DestinationHostname='xmr-jp1.nanopool.org' OR DestinationHostname='xmr-au1.nanopool.org' OR DestinationHostname='xmr.2miners.com' OR DestinationHostname='xmr.hashcity.org' OR DestinationHostname='xmr.f2pool.com' OR DestinationHostname='xmrpool.eu' OR DestinationHostname='p

In [16]:
# `sigma convert -t openobserve -p ../pySigma-backend-openobserve/sigma/pipelines/openobserve/kunai.yml ../sigma/rules/linux/process_creation/proc_creation_lnx_doas_execution.yml`
def convert_rulesdir(
    openobserve_prov,
    rulesdir,
    pipeline_f,
    table_name,
    stats=False,
    limit=5,
    verbosity=0,
):
    """
    Convert sigma rules from rulesdir with given pipeline and openobserve table, Run them
    Return results shape rules list, empty results rules lists, errors rules list
    """
    # rule_f => empty bool, errors true, lines returned
    df_results = pd.DataFrame(columns=["rule", "empty", "errors", "hits"])
    df_stats = pd.DataFrame()
    for rule_f in pathlib.Path(rulesdir).iterdir():
        rule_s = rule_f.name
        if not rule_f.is_file():
            print(f"Skipping rule file {rule_s}")
            continue

        cmd = f"sigma convert -t openobserve -p {pipeline_f} {rule_f}"
        print(f"=> Processing rule file {rule_s}:")
        rule_sql = check_output(
            ["sigma", "convert", "-t", "openobserve", "-p", pipeline_f, rule_f],
            encoding="UTF-8",
        )
        rule_sql = str(rule_sql).replace("<TABLE_NAME>", f'"{table_name}"')
        print(f"```{rule_sql}```")
        try:
            df = openobserve_prov.exec_query(
                rule_sql,
                days=1,
            )
        except:
            df_results.loc[len(df_results)] = [rule_s, False, True, 0]
            continue

        if df.empty:
            df_results.loc[len(df_results)] = [rule_s, True, False, 0]
        else:
            df_results.loc[len(df_results)] = [rule_s, False, False, df.shape[0]]
            if stats:
                try:
                    # print(f"Calculating stats for {rule_s}")
                    df_stats_tmp = rulestats_agg(
                        rule_s,
                        rule_sql,
                        openobserve_prov,
                        1,
                        limit=limit,
                        verbosity=verbosity,
                    )
                    df_stats = pd.concat([df_stats, df_stats_tmp])
                    # print(f"  -> {df_stats_tmp.shape}, {df_stats.shape}")
                except Exception as err:
                    print(f"stats exception: {err}")

    return (df_results, df_stats)

In [17]:
def rulestats_agg(rulename, rule_sql, openobserve_prov, days, limit=5, verbosity=0):
    """
    Make an aggregate count to identify Top results that are possibly noise
    https://sqlglot.com/sqlglot.html#examples
    """
    import sqlglot

    column_names = []

    for column in sqlglot.parse_one(rule_sql).find_all(sqlglot.exp.Column):
        # print(column.alias_or_name)
        column_names.append(column.alias_or_name)
    column_names.append("info_task_name")
    column_names.append("info_parent_task_name")
    column_names = list(set(column_names))
    # print(column_names)
    # Make aggregated query
    agg_sql = (
        rule_sql.replace(
            "SELECT * ", f"SELECT {",".join(column_names)},count(*) as count_ "
        )
        + f" GROUP BY {",".join(column_names)} ORDER BY count_ DESC LIMIT {limit}"
    )
    # only works if len(column_names)=1
    # agg_sql = f"SELECT string_agg({",".join(column_names)}, ', ') as topN FROM (" + test_sql.replace("SELECT * ", f"SELECT {",".join(column_names)},count(*) as count_ ") + f" GROUP BY {",".join(column_names)} ORDER BY count_ DESC LIMIT {topN}" + ')'
    # agg_sql = f"SELECT string_agg(data_exe_path, ', ') as topN FROM (" + test_sql.replace("SELECT * ", f"SELECT {",".join(column_names)},count(*) as count_ ") + f" GROUP BY {",".join(column_names)} ORDER BY count_ DESC LIMIT {topN}" + ')'
    # print(agg_sql)

    df_agg = openobserve_prov.exec_query(agg_sql, days=days, verbosity=verbosity)
    df_agg["rule"] = rulename
    return df_agg

In [18]:
def convert_rule2alert(
    openobserve, rule_f, pipeline_f, table_name, alertowner, alertdestination
):
    """
    Convert given sigma rule to openobserve alert and set it in live instance
    Return results shape rules list, empty results rules lists, errors rules list
    """
    cmd = f"sigma convert -t openobserve -f o2alert -p {pipeline_f} {rule_f}"
    print(f"=> Processing rule file {rule_f}:")
    rule_alert = check_output(
        [
            "sigma",
            "convert",
            "-t",
            "openobserve",
            "-f",
            "o2alert",
            "-p",
            pipeline_f,
            rule_f,
        ],
        encoding="UTF-8",
    )
    rule_alert = str(rule_alert).replace("<TABLE_NAME>", table_name)
    rule_alert = str(rule_alert).replace("<alert-owner-TBD>", alertowner)
    rule_alert = str(rule_alert).replace("<alert-destination-TBD>", alertdestination)
    print(f"```{rule_alert}```")

    openobserve.import_objects_split(
        "alerts",
        json.loads(rule_alert)[0],
        "",
        verbosity=5,
    )

## Functions tests

In [19]:
# test_sql = """SELECT * FROM "kunai" WHERE (data_path LIKE '/etc/cron.d/%' OR data_path LIKE '/etc/cron.daily/%' OR data_path LIKE '/etc/cron.hourly/%' OR data_path LIKE '/etc/cron.monthly/%' OR data_path LIKE '/etc/cron.weekly/%' OR data_path LIKE '/var/spool/cron/crontabs/%') OR (data_path LIKE '%/etc/cron.allow%' OR data_path LIKE '%/etc/cron.deny%' OR data_path LIKE '%/etc/crontab%')"""
# test_sql = """SELECT * FROM "kunai" WHERE data_exe_path LIKE '%/bin/bash' AND (NOT (data_dst_ip='127.0.0.1' OR data_dst_ip='0.0.0.0'))"""
test_sql = """SELECT * FROM "kunai" WHERE (data_exe_path LIKE '%/file' AND regexp_like(data_command_line, '(.){200,}', 'i')) OR (data_exe_path LIKE '%/ls' AND data_command_line LIKE '%-R%') OR data_exe_path LIKE '%/find' OR data_exe_path LIKE '%/tree' OR data_exe_path LIKE '%/findmnt' OR data_exe_path LIKE '%/mlocate'"""

In [20]:
rulestats_agg("rulename", test_sql, openobserve_prov, 1)

,count_,data_command_line,data_exe_path,info_parent_task_name,info_task_name,rule
0,624,find /usr/lib/php -mindepth 1 -maxdepth 1 -reg...,/usr/bin/find,phpquery,find,rulename
1,576,find /var/log/zeek -type d -regex .*/[0-9][0-9...,/usr/bin/find,bash,find,rulename
2,575,find /var/log/zeek -type f -regex .*/[0-9][0-9...,/usr/bin/find,bash,find,rulename
3,528,find -O3 /var/lib/php/sessions/ -ignore_readdi...,/usr/bin/find,sessionclean,find,rulename
4,192,find /proc/231/fd -ignore_readdir_race -lname ...,/usr/bin/find,sessionclean,find,rulename


## Rules Conversion
Convert rules, check against live instance if returns results, errors or not.
Include or not statistics of TopN aggregated result for matching rule.

In [21]:
rulesdir = os.environ["HOME"] + "/tmp/sigma/rules/linux/network_connection"

In [22]:
(df_res, df_stats) = convert_rulesdir(
    openobserve_prov, rulesdir, pipeline_f, table_name, stats=True
)

=> Processing rule file net_connection_lnx_crypto_mining_indicators.yml:


Parsing Sigma rules


```SELECT * FROM "kunai" WHERE data_dst_hostname='pool.minexmr.com' OR data_dst_hostname='fr.minexmr.com' OR data_dst_hostname='de.minexmr.com' OR data_dst_hostname='sg.minexmr.com' OR data_dst_hostname='ca.minexmr.com' OR data_dst_hostname='us-west.minexmr.com' OR data_dst_hostname='pool.supportxmr.com' OR data_dst_hostname='mine.c3pool.com' OR data_dst_hostname='xmr-eu1.nanopool.org' OR data_dst_hostname='xmr-eu2.nanopool.org' OR data_dst_hostname='xmr-us-east1.nanopool.org' OR data_dst_hostname='xmr-us-west1.nanopool.org' OR data_dst_hostname='xmr-asia1.nanopool.org' OR data_dst_hostname='xmr-jp1.nanopool.org' OR data_dst_hostname='xmr-au1.nanopool.org' OR data_dst_hostname='xmr.2miners.com' OR data_dst_hostname='xmr.hashcity.org' OR data_dst_hostname='xmr.f2pool.com' OR data_dst_hostname='xmrpool.eu' OR data_dst_hostname='pool.hashvault.pro' OR data_dst_hostname='moneroocean.stream' OR data_dst_hostname='monerocean.stream'
```
=> Processing rule file net_connection_lnx_back_connect

Parsing Sigma rules


```SELECT * FROM "kunai" WHERE data_exe_path LIKE '%/bin/bash' AND (NOT (data_dst_ip='127.0.0.1' OR data_dst_ip='0.0.0.0'))
```
=> Processing rule file net_connection_lnx_domain_localtonet_tunnel.yml:


...

```SELECT * FROM "kunai" WHERE (data_dst_hostname LIKE '%.localto.net' OR data_dst_hostname LIKE '%.localtonet.com') AND Initiated='true'
```
=> Processing rule file net_connection_lnx_susp_malware_callback_port.yml:


Parsing Sigma rules


```SELECT * FROM "kunai" WHERE (Initiated='true' AND (data_dst_port=888 OR data_dst_port=999 OR data_dst_port=2200 OR data_dst_port=2222 OR data_dst_port=4000 OR data_dst_port=4444 OR data_dst_port=6789 OR data_dst_port=8531 OR data_dst_port=50501 OR data_dst_port=51820)) AND (NOT (data_dst_ip LIKE '127.%' OR data_dst_ip LIKE '10.%' OR data_dst_ip LIKE '172.16.%' OR data_dst_ip LIKE '172.17.%' OR data_dst_ip LIKE '172.18.%' OR data_dst_ip LIKE '172.19.%' OR data_dst_ip LIKE '172.20.%' OR data_dst_ip LIKE '172.21.%' OR data_dst_ip LIKE '172.22.%' OR data_dst_ip LIKE '172.23.%' OR data_dst_ip LIKE '172.24.%' OR data_dst_ip LIKE '172.25.%' OR data_dst_ip LIKE '172.26.%' OR data_dst_ip LIKE '172.27.%' OR data_dst_ip LIKE '172.28.%' OR data_dst_ip LIKE '172.29.%' OR data_dst_ip LIKE '172.30.%' OR data_dst_ip LIKE '172.31.%' OR data_dst_ip LIKE '192.168.%' OR data_dst_ip LIKE '169.254.%' OR data_dst_ip='::1/128' OR data_dst_ip LIKE 'fe8%' OR data_dst_ip LIKE 'fe9%' OR data_dst_ip LIKE 'fea%'

Parsing Sigma rules


```SELECT * FROM "kunai" WHERE data_dst_hostname LIKE '%tunnel.us.ngrok.com%' OR data_dst_hostname LIKE '%tunnel.eu.ngrok.com%' OR data_dst_hostname LIKE '%tunnel.ap.ngrok.com%' OR data_dst_hostname LIKE '%tunnel.au.ngrok.com%' OR data_dst_hostname LIKE '%tunnel.sa.ngrok.com%' OR data_dst_hostname LIKE '%tunnel.jp.ngrok.com%' OR data_dst_hostname LIKE '%tunnel.in.ngrok.com%'
```


In [23]:
df_res

,rule,empty,errors,hits
0,net_connection_lnx_crypto_mining_indicators.yml,True,False,0
1,net_connection_lnx_back_connect_shell_dev.yml,True,False,0
2,net_connection_lnx_domain_localtonet_tunnel.yml,False,True,0
3,net_connection_lnx_susp_malware_callback_port.yml,False,True,0
4,net_connection_lnx_ngrok_tunnel.yml,True,False,0


In [24]:
df_stats

""


In [25]:
rulesdir = os.environ["HOME"] + "/tmp/sigma/rules/linux/file_event"
(df_res3, df_stats3) = convert_rulesdir(
    openobserve_prov, rulesdir, pipeline_f, table_name, stats=True
)

=> Processing rule file file_event_lnx_susp_shell_script_under_profile_directory.yml:


Parsing Sigma rules


```SELECT * FROM "kunai" WHERE data_path LIKE '%/etc/profile.d/%' AND (data_path LIKE '%.csh' OR data_path LIKE '%.sh')
```
=> Processing rule file file_event_lnx_wget_download_file_in_tmp_dir.yml:


Parsing Sigma rules


```SELECT * FROM "kunai" WHERE data_exe_path LIKE '%/wget' AND (data_path LIKE '/tmp/%' OR data_path LIKE '/var/tmp/%')
```
=> Processing rule file file_event_lnx_triple_cross_rootkit_persistence.yml:


...

```SELECT * FROM "kunai" WHERE data_path LIKE '%/etc/doas.conf'
```
=> Processing rule file file_event_lnx_triple_cross_rootkit_lock_file.yml:


Parsing Sigma rules


```SELECT * FROM "kunai" WHERE data_path='/tmp/rootlog'
```
=> Processing rule file file_event_lnx_persistence_cron_files.yml:


Parsing Sigma rules


```SELECT * FROM "kunai" WHERE (data_path LIKE '/etc/cron.d/%' OR data_path LIKE '/etc/cron.daily/%' OR data_path LIKE '/etc/cron.hourly/%' OR data_path LIKE '/etc/cron.monthly/%' OR data_path LIKE '/etc/cron.weekly/%' OR data_path LIKE '/var/spool/cron/crontabs/%') OR (data_path LIKE '%/etc/cron.allow%' OR data_path LIKE '%/etc/cron.deny%' OR data_path LIKE '%/etc/crontab%')
```


In [26]:
df_res3

,rule,empty,errors,hits
0,file_event_lnx_susp_shell_script_under_profile...,False,False,271
1,file_event_lnx_wget_download_file_in_tmp_dir.yml,True,False,0
2,file_event_lnx_triple_cross_rootkit_persistenc...,True,False,0
3,file_event_lnx_persistence_sudoers_files.yml,False,False,1000
4,file_event_lnx_doas_conf_creation.yml,True,False,0
5,file_event_lnx_triple_cross_rootkit_lock_file.yml,True,False,0
6,file_event_lnx_persistence_cron_files.yml,False,False,800


In [27]:
df_stats3

,count_,data_path,info_parent_task_name,info_task_name,rule
0,20,/etc/profile.d/bash_completion.sh,sshd,bash,file_event_lnx_susp_shell_script_under_profile...
1,19,/etc/profile.d/01-locale-fix.sh,sshd,bash,file_event_lnx_susp_shell_script_under_profile...
2,18,/etc/profile.d/gawk.sh,sshd,bash,file_event_lnx_susp_shell_script_under_profile...
3,17,/etc/profile.d/npm.sh,sshd,bash,file_event_lnx_susp_shell_script_under_profile...
4,17,/etc/profile.d/Z99-cloud-locale-test.sh,sshd,bash,file_event_lnx_susp_shell_script_under_profile...
0,2070,/etc/sudoers.d/README,sh,sudo,file_event_lnx_persistence_sudoers_files.yml
1,1620,/etc/sudoers.d/aide-unprivileged,sh,sudo,file_event_lnx_persistence_sudoers_files.yml
2,1620,/etc/sudoers.d/zfs,sh,sudo,file_event_lnx_persistence_sudoers_files.yml
3,1619,/etc/sudoers.d/backup_lab,sh,sudo,file_event_lnx_persistence_sudoers_files.yml
4,1619,/etc/sudoers.d/90-cloud-init-users,sh,sudo,file_event_lnx_persistence_sudoers_files.yml


In [28]:
rulesdir = os.environ["HOME"] + "/tmp/sigma/rules/linux/process_creation"
(df_res2, df_stats2) = convert_rulesdir(
    openobserve_prov, rulesdir, pipeline_f, table_name, stats=True
)

=> Processing rule file proc_creation_lnx_malware_gobrat_grep_payload_discovery.yml:


Parsing Sigma rules


```SELECT * FROM "kunai" WHERE data_exe_path LIKE '%/grep' AND (data_command_line LIKE '%apached%' OR data_command_line LIKE '%frpc%' OR data_command_line LIKE '%sshd.sh%' OR data_command_line LIKE '%zone.arm%')
```
=> Processing rule file proc_creation_lnx_touch_susp.yml:


Parsing Sigma rules


```SELECT * FROM "kunai" WHERE data_exe_path LIKE '%/touch' AND data_command_line LIKE '% -t %' AND data_command_line LIKE '%.service'
```
=> Processing rule file proc_creation_lnx_file_and_directory_discovery.yml:


...

```SELECT * FROM "kunai" WHERE (data_exe_path LIKE '%/esxcli' AND data_command_line LIKE '%vsan%') AND (data_command_line LIKE '% get%' OR data_command_line LIKE '% list%')
```
=> Processing rule file proc_creation_lnx_remove_package.yml:


Parsing Sigma rules


```SELECT * FROM "kunai" WHERE (data_exe_path LIKE '%/yum' AND (data_command_line LIKE '%erase%' OR data_command_line LIKE '%remove%')) OR ((data_exe_path LIKE '%/apt' OR data_exe_path LIKE '%/apt-get') AND (data_command_line LIKE '%remove%' OR data_command_line LIKE '%purge%')) OR (data_exe_path LIKE '%/dpkg' AND (data_command_line LIKE '%--remove %' OR data_command_line LIKE '% -r %')) OR (data_exe_path LIKE '%/rpm' AND data_command_line LIKE '% -e %')
```
=> Processing rule file proc_creation_lnx_clipboard_collection.yml:


Parsing Sigma rules


```SELECT * FROM "kunai" WHERE data_exe_path LIKE '%xclip%' AND (data_command_line LIKE '%-sel%' AND data_command_line LIKE '%clip%' AND data_command_line LIKE '%-o%')
```


In [29]:
df_res2

,rule,empty,errors,hits
0,proc_creation_lnx_malware_gobrat_grep_payload_...,True,False,0
1,proc_creation_lnx_touch_susp.yml,True,False,0
2,proc_creation_lnx_file_and_directory_discovery...,False,False,1000
3,proc_creation_lnx_ssm_agent_abuse.yml,True,False,0
4,proc_creation_lnx_base64_shebang_cli.yml,True,False,0
...,...,...,...,...
108,proc_creation_lnx_crontab_removal.yml,True,False,0
109,proc_creation_lnx_omigod_scx_runasprovider_exe...,False,True,0
110,proc_creation_lnx_esxcli_vsan_discovery.yml,True,False,0
111,proc_creation_lnx_remove_package.yml,False,False,101


In [30]:
df_stats2

,count_,data_command_line,data_exe_path,info_parent_task_name,info_task_name,rule
0,624,find /usr/lib/php -mindepth 1 -maxdepth 1 -reg...,/usr/bin/find,phpquery,find,proc_creation_lnx_file_and_directory_discovery...
1,576,find /var/log/zeek -type d -regex .*/[0-9][0-9...,/usr/bin/find,bash,find,proc_creation_lnx_file_and_directory_discovery...
2,575,find /var/log/zeek -type f -regex .*/[0-9][0-9...,/usr/bin/find,bash,find,proc_creation_lnx_file_and_directory_discovery...
3,528,find -O3 /var/lib/php/sessions/ -ignore_readdi...,/usr/bin/find,sessionclean,find,proc_creation_lnx_file_and_directory_discovery...
4,192,find /proc/231/fd -ignore_readdir_race -lname ...,/usr/bin/find,sessionclean,find,proc_creation_lnx_file_and_directory_discovery...
...,...,...,...,...,...,...
4,27,id -u,/usr/bin/id,update-motd-fsc,id,proc_creation_lnx_local_account.yml
0,2,chattr -R -i /lib /etc/init.d /boot,/usr/bin/chattr,sh,chattr,proc_creation_lnx_chattr_immutable_removal.yml
0,45,apt remove libssl1.0.0,/usr/bin/apt,bash,apt,proc_creation_lnx_remove_package.yml
1,42,apt remove libssl1.1,/usr/bin/apt,bash,apt,proc_creation_lnx_remove_package.yml


## Sigma rule to alerts

In [ ]:
# for all rules with no results on given timeperiod, let's put alert
# `sigma convert -t openobserve -f o2alert -p ../pySigma-backend-openobserve/sigma/pipelines/openobserve/kunai.yml ../sigma/rules/linux/process_creation/proc_creation_lnx_doas_execution.yml`
# replace <TABLE_NAME>, '<alert-owner-TBD>

In [ ]:
# SELECT * FROM "kunai" WHERE data_exe_path LIKE '%/doas'
res_to_alerts = [
    "/home/user/tmp/sigma/rules/linux/process_creation/proc_creation_lnx_doas_execution.yml"
]
# res_to_alerts = ['/home/user/tmp/sigma/rules/linux/network_connection/net_connection_lnx_back_connect_shell_dev.yml']

In [ ]:
# FIXME! backend issue on alert importing
#   'Return 400. Text: Json deserialize error: Failed to decode at <EOF>
for rule_f in res_to_alerts:
    convert_rule2alert(
        OO, rule_f, pipeline_f, "kunai", "root@example.com", "alert-destination-email"
    )

## References

- [OpenObserve REST API](https://openobserve.ai/docs/api/)
- [Unofficial python-openobserve module](https://github.com/JustinGuese/python-openobserve) with some customizations and enhancements (>=0.2)
- Openobserve github discussions: https://github.com/openobserve/openobserve/discussions/